### Parsing

In [1]:
import string
import itertools
import numpy as np
from tqdm import tqdm
from collections import deque, namedtuple, OrderedDict

Entity = namedtuple("Entity", ["position", "id", "kind", "attack", "hp"])

def parse(text):
    ids = {"E": itertools.cycle(string.ascii_lowercase), "G": itertools.cycle(string.ascii_uppercase)}
    arr = np.array([[c for c in line.strip()] for line in text.splitlines() if line])
    entities = [
        Entity(position=tuple(pos), id=next(ids[kind]), kind=kind, attack=3, hp=200)
        for kind in "EG"
        for pos in np.argwhere(arr == kind)
    ]
    for entity in entities:
        arr[entity.position] = "."
    return arr, entities

parse("""
#####
#G..#
##G.#
#E..#
#####
""")

(array([['#', '#', '#', '#', '#'],
        ['#', '.', '.', '.', '#'],
        ['#', '#', '.', '.', '#'],
        ['#', '.', '.', '.', '#'],
        ['#', '#', '#', '#', '#']], 
       dtype='<U1'),
 [Entity(position=(3, 1), id='a', kind='E', attack=3, hp=200),
  Entity(position=(1, 1), id='A', kind='G', attack=3, hp=200),
  Entity(position=(2, 2), id='B', kind='G', attack=3, hp=200)])

### Pathfinding

In [37]:
def board(arr, entities, use_id=False):
    arr = arr.copy()
    for entity in entities:
        arr[entity.position] = entity.id if use_id else entity.kind
    return arr

def get_shortest_path(start, end, arr):
    """ breadth-first search adapted from https://stackoverflow.com/a/47902476 """
    arr = arr.copy()
    m, n = arr.shape
    start, end = tuple(start), tuple(end)
    arr[end] = "*"
    queue = deque([[start]])
    seen = set([start])
    while queue:
        path = queue.popleft()
        (r, c) = path[-1]
        if (r, c) == end:
            return path[1:]
        for r2, c2 in ((r - 1, c), (r, c - 1), (r, c + 1), (r + 1, c)):
            if 0 <= c2 < n and 0 <= r2 < n and arr[r2, c2] in ".*" and (r2, c2) not in seen:
                queue.append(path + [(r2, c2)])
                seen.add((r2, c2))

def paths_to_enemies(entity, enemies, arr):
    mask = board(arr, enemies)
    paths = []
    for enemy in enemies:
        paths.append(get_shortest_path(entity.position, enemy.position, arr))
    # remove impossible ones
    paths = [path for path in paths if path is not None]
    # sort by number of steps (min=1) and book order of target
    return sorted(paths, key=lambda x: (len(x), x[-1]))

arr, _ = parse("""
#####
#E..#
##.##
#G.G#
#####
""")
get_shortest_path((1, 1), (3, 3), arr)

[(1, 2), (2, 2), (3, 2), (3, 3)]

### Gameplay

In [38]:
def move(entity, new_position):
    return Entity(new_position, *entity[1:])

def attack(attacker, opponent):
    new_hp = opponent.hp - attacker.attack
    opponent = Entity(
        position=opponent.position,
        id=opponent.id,
        kind=opponent.kind,
        attack=opponent.attack,
        hp=new_hp
    )
    return attacker, opponent

def game_over(entities):
    return all([e.kind == entities[0].kind for e in entities])

def turn(arr, entities, verbose=False):
    entities = sorted(entities)
    
    # create a mutable state holder separate from the list we will iterate over
    state = {e.position: e for e in entities}
    
    for player in entities:
        if game_over(list(state.values())):
            return list(state.values()), 0
        
        if player.position not in state:
            # this player must have been killed
            continue
        
        # figure out who is closest
        player = state.pop(player.position)
        enemies = [e for _, e in state.items() if e.kind != player.kind]
        
        # either choose attack target right away or try to move
        paths = paths_to_enemies(player, enemies, board(arr, state.values()))
        adjacent = [path[-1] for path in paths if len(path) == 1]
        if not adjacent:
            if not paths:
                state[player.position] = player
                continue  # we can't do anything
            # move
            path, target_pos = paths[0], paths[0][-1]
            player = move(player, path[0])
            if verbose:
                print(f"{player.id} moves towards {state[target_pos].id}: step to {path[0]}")
        
        # if there is still nobody adjacent, finish up
        paths = paths_to_enemies(player, enemies, board(arr, state.values()))
        adjacent = [path[-1] for path in paths if len(path) == 1]
        if not adjacent:
            state[player.position] = player
            continue
            
        if adjacent:
            # attack enemy with lowest HP
            attacking = sorted(adjacent, key=lambda pos: (state[pos].hp, pos))[0]
            opponent = state.pop(attacking)
            if verbose:
                print(f"{player.id} attacks {opponent.id} at {attacking}")
            player, opponent = attack(player, opponent)
            state[player.position] = player
            if opponent.hp > 0:
                state[opponent.position] = opponent
            elif verbose:
                print(f"-- opponent {opponent.id} died")

    return list(state.values()), 1

def pretty_print(arr, entities, use_id=False):
    arr = board(arr, entities, use_id=use_id)
    for i, line in enumerate(arr):
        entities_in_row = sorted([e for e in entities if e.position[0] == i])
        extra = ", ".join([f"{e.id}({e.hp})" for e in entities_in_row])
        print("".join(line) + "   " + extra)
        
def play(arr, entities, verbose=False, max_rounds=10_000):
    if verbose:
        print("After 0 rounds:")
        pretty_print(arr, entities)
    i = 0
    while i < max_rounds and not game_over(entities):
        entities, full = turn(arr, entities, verbose=verbose)
        i += full
        if verbose:
            print()
            print(f"After {i} rounds:")
            pretty_print(arr, entities)
            print()

    return i, entities
    
arr, entities = parse("""
####
#G.#
#.E#
####
""")
play(arr, entities, verbose=True, max_rounds=3)

After 0 rounds:
####   
#G.#   A(200)
#.E#   a(200)
####   
A moves towards a: step to (1, 2)
A attacks a at (2, 2)
a attacks A at (1, 2)

After 1 rounds:
####   
#.G#   A(197)
#.E#   a(197)
####   

A attacks a at (2, 2)
a attacks A at (1, 2)

After 2 rounds:
####   
#.G#   A(194)
#.E#   a(194)
####   

A attacks a at (2, 2)
a attacks A at (1, 2)

After 3 rounds:
####   
#.G#   A(191)
#.E#   a(191)
####   



(3,
 [Entity(position=(2, 2), id='a', kind='E', attack=3, hp=191),
  Entity(position=(1, 2), id='A', kind='G', attack=3, hp=191)])

### Part 1

In [39]:
def outcome(text):
    text = "\n".join(["".join(line.split(" ")[0].strip()) for line in text.splitlines() if line.startswith("#")])
    arr, entities = parse(text)
    n_rounds, result = play(arr, entities, verbose=False)
    hp_left = sum(entity.hp for entity in result)
    answer = hp_left * n_rounds
    
    pretty_print(arr, result)
    print(f"Combat ends after {n_rounds} full rounds")
    print(f"{entities[0].kind} win with {hp_left} total hit points left")
    print(f"Outcome: {n_rounds} * {hp_left} = {answer}")
    return answer

In [40]:
assert outcome("""
#######
#.G...#
#...EG#
#.#.#G#
#..G#E#
#.....#
#######
""") == 27730

#######   
#G....#   A(200)
#.G...#   B(131)
#.#.#G#   C(59)
#...#.#   
#....G#   D(200)
#######   
Combat ends after 47 full rounds
E win with 590 total hit points left
Outcome: 47 * 590 = 27730


In [41]:
assert outcome("""
#######       #######
#G..#E#       #...#E#   E(200)
#E#E.E#       #E#...#   E(197)
#G.##.#  -->  #.E##.#   E(185)
#...#E#       #E..#E#   E(200), E(200)
#...E.#       #.....#
#######       #######
Combat ends after 37 full rounds
Elves win with 982 total hit points left
Outcome: 37 * 982 = 36334
""") == 36334

#######   
#...#E#   a(200)
#E#...#   c(197)
#.E##.#   f(185)
#E..#E#   e(200), d(200)
#.....#   
#######   
Combat ends after 37 full rounds
E win with 982 total hit points left
Outcome: 37 * 982 = 36334


In [42]:
assert outcome("""
#######       #######   
#E..EG#       #.E.E.#   E(164), E(197)
#.#G.E#       #.#E..#   E(200)
#E.##E#  -->  #E.##.#   E(98)
#G..#.#       #.E.#.#   E(200)
#..E#.#       #...#.#   
#######       #######   
Combat ends after 46 full rounds
Elves win with 859 total hit points left
Outcome: 46 * 859 = 39514
""") == 39514

#######   
#.E.E.#   a(164), c(197)
#.#E..#   e(200)
#E.##.#   d(98)
#.E.#.#   f(200)
#...#.#   
#######   
Combat ends after 46 full rounds
E win with 859 total hit points left
Outcome: 46 * 859 = 39514


In [43]:
assert outcome("""
#######       #######   
#E.G#.#       #G.G#.#   G(200), G(98)
#.#G..#       #.#G..#   G(200)
#G.#.G#  -->  #..#..#   
#G..#.#       #...#G#   G(95)
#...E.#       #...G.#   G(200)
#######       #######  
Combat ends after 35 full rounds
Goblins win with 793 total hit points left
Outcome: 35 * 793 = 27755
""") == 27755

#######   
#G.G#.#   C(200), A(98)
#.#G..#   B(200)
#..#..#   
#...#G#   D(95)
#...G.#   E(200)
#######   
Combat ends after 35 full rounds
E win with 793 total hit points left
Outcome: 35 * 793 = 27755


In [44]:
assert outcome("""
#######       #######   
#.E...#       #.....#   
#.#..G#       #.#G..#   G(200)
#.###.#  -->  #.###.#   
#E#G#G#       #.#.#.#   
#...#G#       #G.G#G#   G(98), G(38), G(200)
#######       #######   
""") == 28944

#######   
#.....#   
#.#G..#   C(200)
#.###.#   
#.#.#.#   
#G.G#G#   A(98), B(38), D(200)
#######   
Combat ends after 54 full rounds
E win with 536 total hit points left
Outcome: 54 * 536 = 28944


In [45]:
assert outcome("""
#########       #########   
#G......#       #.G.....#   G(137)
#.E.#...#       #G.G#...#   G(200), G(200)
#..##..G#       #.G##...#   G(200)
#...##..#  -->  #...##..#   
#...#...#       #.G.#...#   G(200)
#.G...G.#       #.......#   
#.....G.#       #.......#   
#########       ######### 
Combat ends after 20 full rounds
Goblins win with 937 total hit points left
Outcome: 20 * 937 = 18740
""") == 18740

#########   
#.G.....#   A(137)
#G.G#...#   D(200), B(200)
#.G##...#   C(200)
#...##..#   
#.G.#...#   E(200)
#.......#   
#.......#   
#########   
Combat ends after 20 full rounds
E win with 937 total hit points left
Outcome: 20 * 937 = 18740


In [ ]:
with open("../inputs/15/input.txt") as fp:
    output(fp.read())